<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/Kalman/PermianRigInput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install filterpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 7.9 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=61fcfa4d13586708a7b1572e9a676959d418de0fce5e6e51e695b50d101b1437
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
Successfully built filterpy


In [3]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.stats import plot_covariance_ellipse
from filterpy.kalman import predict
from filterpy.kalman import update
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score 

In [4]:
from google.colab import files
uploaded = files.upload() # gridData4Hiro_EFandPerm.xlsx

Saving DPR42.csv to DPR42.csv


In [5]:
data = pd.read_csv('DPR42.csv')

In [6]:
data.tail()

,Step,Month,BH,dBH,BH2msf,rgr,PR,NPR,NPRr,DPR,DPRr
150,150,7/1/2019,441,-7,455,0.984848,4342565,356431,0.082078,-287937,-0.066306
151,151,8/1/2019,438,-13,442,0.971429,4463443,398337,0.089244,-277459,-0.062163
152,152,9/1/2019,419,-1,441,0.997738,4548306,455017,0.100041,-370153,-0.081383
153,153,10/1/2019,418,-3,438,0.993197,4633550,395996,0.085463,-310752,-0.067066
154,154,11/1/2019,408,-19,419,0.956621,4768876,430530,0.090279,-295204,-0.061902


In [7]:
# Step units in months
dt = 1

# x - Origin state estimate vector
x = np.array([840000, -0.009, 0.009]).T

# Q - Process noise matrix
Q = Q_discrete_white_noise(dim=3, dt=1., var=1.12)

# P - Covariance matrix
P = np.diag([100, 1, 1])

# R - Measurement noise matrix
R = np.diag([26, -80, 32])

# H - Measurement function
H = np.array([[1., 0., 0.],
              [0., 1., 0.],
              [0., 0., 1.]])

# F - State transition matrix
F = np.array([[1, 0.85*dt, 1.1*dt],
              [0, 1, 0],
              [0, 0 , 1]])

# Measurements for update 
zsu = data[['PR','DPR','NPR','BH2msf']].to_numpy()

# System prediction 
Xp = np.empty((0,3), int)

# System update
Xu = np.empty((0,3), int)
for zs  in zsu:
    rgr = zs[3:4]
    z = zs[0:3]
    u = zs[1:4]
 
    u[0]=0
    u[1]=0

    x, P = predict(x=x, P=P, F=F, Q=Q)    #x, P = predict(x=x, P=P, F=F, Q=Q, u=u)
    Xp = np.vstack([Xp, x])

    x, P = update(x, P, z, R, H)
    Xu = np.vstack([Xu, x])
     
r2_score(Xp[:,0], data.PR)     

0.9987850907727607

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,0], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.PR, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,0], mode='lines+markers',name='UpdatePR'))
fig.show()

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,1], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.DPR, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,1], mode='lines+markers',name='UpdatePR'))
fig.show()

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,2], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.NPR, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,2], mode='lines+markers',name='UpdatePR'))
fig.show()